In [32]:
#Import neccesary libraries
import sqlite3
import pandas as pd
import numpy as np

In [8]:
#Connect to sqlite file
db = sqlite3.connect('exercise01.sqlite')

In [41]:
#SQL query to flatten database into single file
sql_flatten = """
SELECT records.id, age, 
countries.name as "country", education_levels.name as "education_level", education_num,
workclasses.name as "workclass", marital_statuses.name as "marital_status",
occupations.name as "occupation", races.name as "race", relationships.name as "relationship_status",
sexes.name as "sex",capital_gain, capital_loss,  hours_week, over_50k
FROM records
JOIN countries ON records.country_id = countries.id
JOIN education_levels ON records.education_level_id = education_levels.id
JOIN workclasses ON records.workclass_id = workclasses.id
JOIN marital_statuses ON records.marital_status_id = marital_statuses.id
JOIN occupations ON records.occupation_id = occupations.id
JOIN races ON records.race_id = races.id
JOIN relationships ON records.relationship_id = relationships.id
JOIN sexes ON records.sex_id = sexes.id
"""

flattened = pd.read_sql_query(sql_flatten, db, index_col = "id")
flattened.head()

,age,country,education_level,education_num,workclass,marital_status,occupation,race,relationship_status,sex,capital_gain,capital_loss,hours_week,over_50k
id,,,,,,,,,,,,,,
1,39,United-States,Bachelors,13,State-gov,Never-married,Adm-clerical,White,Not-in-family,Male,2174,0,40,0
2,50,United-States,Bachelors,13,Self-emp-not-inc,Married-civ-spouse,Exec-managerial,White,Husband,Male,0,0,13,0
3,38,United-States,HS-grad,9,Private,Divorced,Handlers-cleaners,White,Not-in-family,Male,0,0,40,0
4,53,United-States,11th,7,Private,Married-civ-spouse,Handlers-cleaners,Black,Husband,Male,0,0,40,0
5,28,Cuba,Bachelors,13,Private,Married-civ-spouse,Prof-specialty,Black,Wife,Female,0,0,40,0


In [33]:
#Check query didnt change size of records, should be 0,0
records = pd.read_sql_query("SELECT * FROM records", db)
np.subtract(flattened.shape,records.shape)


array([0, 0])

In [42]:
#write flattened database to csv file in local directory, using the id column as index
flattened.to_csv("flattened.csv",index_label = "id" )

In [46]:
#read the csv file into a pandas dataframe
flat_df = pd.read_csv("flattened.csv", index_col = 0)
flat_df.head()

,age,country,education_level,education_num,workclass,marital_status,occupation,race,relationship_status,sex,capital_gain,capital_loss,hours_week,over_50k
id,,,,,,,,,,,,,,
1,39,United-States,Bachelors,13,State-gov,Never-married,Adm-clerical,White,Not-in-family,Male,2174,0,40,0
2,50,United-States,Bachelors,13,Self-emp-not-inc,Married-civ-spouse,Exec-managerial,White,Husband,Male,0,0,13,0
3,38,United-States,HS-grad,9,Private,Divorced,Handlers-cleaners,White,Not-in-family,Male,0,0,40,0
4,53,United-States,11th,7,Private,Married-civ-spouse,Handlers-cleaners,Black,Husband,Male,0,0,40,0
5,28,Cuba,Bachelors,13,Private,Married-civ-spouse,Prof-specialty,Black,Wife,Female,0,0,40,0


In [47]:
flat_df.describe()

,age,education_num,capital_gain,capital_loss,hours_week,over_50k
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,10.078089,1079.067626,87.502314,40.422382,0.239282
std,13.710510,2.570973,7452.019058,403.004552,12.391444,0.426649
min,17.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000,1.000000
